You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Outline:
I. Introduction
- Brief overview of Artificial Intelligence (AI)
- Importance of staying updated on AI trends and key players

II. Latest Trends in Artificial Intelligence
A. Machine Learning advancements
B. Natural Language Processing breakthroughs
C. Computer Vision innovations

III. Key Players in Artificial Intelligence
A. Google AI
B. IBM Watson
C. Amazon AI
D. Microsoft Cognitive Services

IV. Notewor

I now can give a great answer

Final Answer: 

# Title: The Benefits of Remote Work

In today's fast-paced world, remote work has become increasingly popular among companies and employees alike. The flexibility and freedom that come with working from home or any location of choice have revolutionized the traditional office setting. Remote work allows employees to have a better work-life balance, eliminating the need for long commutes and allowing for more time spent with family and friends. 

Furthermore, remote work can lead to increased productivity as employees are able to work in a comfortable environment that suits their individual needs. With the advancements in technology, communication and collaboration tools have made it easier than ever for remote teams to stay connected and work efficiently. Employers also benefit from remote work by saving on office space and overhead costs. 

Overall, the shift towards remote work is a positive trend that provides numerous benefits for bot

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Title: The Benefits of Remote Work

In today's fast-paced world, remote work has become increasingly popular among companies and employees alike. The flexibility and freedom that come with working from home or any location of choice have revolutionized the traditional office setting. Remote work allows employees to have a better work-life balance, eliminating the need for long commutes and allowing for more time spent with family and friends. 

Furthermore, remote work can lead to increased productivity as employees are able to work in a comfortable environment that suits their individual needs. With the advancements in technology, communication and collaboration tools have made it easier than ever for remote teams to stay connected and work efficiently. Employers also benefit from remote work by saving on office space and overhead costs. 

Overall, the shift towards remote work is a positive trend that provides numerous benefits for both employees and employers. As we continue to navigate the changing landscape of work, embracing remote work can lead to a more productive and satisfied workforce. 

# Conclusion

In conclusion, remote work offers a multitude of advantages that can enhance the way we work and live. By taking advantage of the flexibility and technology available, remote work has the potential to transform the traditional office environment for the better. As we move forward, it is important for companies to embrace remote work as a viable option that can lead to increased productivity and employee satisfaction.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Exploring the Latest Trends in Digital Marketing

Outline:
I. Introduction
- Brief overview of digital marketing
- Importance of staying updated on latest trends

II. Latest Trends in Digital Marketing
A. Video Marketing
- Rise of video content on social media platforms
- Importance of incorporating video into digital marketing strategies
B. Artificial Intelligence
- Use of AI in personalized marketing campaigns
- Impact of AI on customer experience
C. Voice Search Optimization
- Growth of voi

I now can give a great answer

Final Answer:

# Exploring the Latest Trends in Digital Marketing

In today's fast-paced digital landscape, staying updated on the latest trends in digital marketing is crucial for both professionals and businesses. As technology continues to evolve, new opportunities and challenges arise, making it essential to adapt and innovate in order to stay competitive. This article delves into the most recent trends shaping the digital marketing industry, from video marketing to artificial intelligence, and provides valuable insights for readers looking to enhance their strategies.

## Latest Trends in Digital Marketing

### Video Marketing
Video content has become increasingly popular across social media platforms, with platforms like Instagram, TikTok, and YouTube leading the way. The rise of short-form videos and live streaming has revolutionized how brands engage with their audiences, offering a more interactive and immersive experience. Incorporating video in

In [14]:
Markdown(result)

# Exploring the Latest Trends in Digital Marketing

In today's fast-paced digital landscape, staying updated on the latest trends in digital marketing is crucial for both professionals and businesses. As technology continues to evolve, new opportunities and challenges arise, making it essential to adapt and innovate in order to stay competitive. This article delves into the most recent trends shaping the digital marketing industry, from video marketing to artificial intelligence, and provides valuable insights for readers looking to enhance their strategies.

## Latest Trends in Digital Marketing

### Video Marketing
Video content has become increasingly popular across social media platforms, with platforms like Instagram, TikTok, and YouTube leading the way. The rise of short-form videos and live streaming has revolutionized how brands engage with their audiences, offering a more interactive and immersive experience. Incorporating video into digital marketing strategies has become a necessity, as it allows for more creative storytelling and better engagement with consumers.

### Artificial Intelligence
Artificial Intelligence (AI) is transforming the way marketers approach personalized campaigns and customer experiences. By leveraging AI-powered tools, businesses can analyze data more effectively, predict consumer behavior, and deliver targeted content to the right audience at the right time. From chatbots to recommendation engines, AI is reshaping the digital marketing landscape and helping brands drive more meaningful interactions with their customers.

### Voice Search Optimization
With the increasing popularity of voice assistants like Siri, Alexa, and Google Assistant, optimizing content for voice search has become a priority for digital marketers. Voice search usage is on the rise, and businesses need to adapt their SEO strategies to cater to this new trend. By focusing on long-tail keywords, natural language, and providing concise answers to common queries, brands can improve their visibility in voice search results and enhance the overall user experience.

### Influencer Marketing
Influencers play a significant role in digital marketing, as they have the power to sway consumer opinions and drive purchase decisions. Collaborating with influencers can help brands reach a larger audience, build credibility, and create authentic connections with consumers. By partnering with influencers who align with their values and target audience, businesses can leverage their influence to boost brand awareness and generate leads.

Stay ahead of the curve by implementing these latest digital marketing trends in your strategies. Keep learning and adapting to stay competitive in the ever-evolving digital landscape.

Sources:
- HubSpot
- Moz
- Social Media Examiner
- Digital Marketing Institute

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).